In [ ]:
# 🐆 Jaguar Re-Identification Challenge - Complete Solution Notebook

Below is a comprehensive, professional notebook for the Jaguar Re-Identification Challenge. I've structured it with clear sections, explanations, and visualizations.

---

## 📋 Table of Contents

1. [Introduction & Author Info](#1-introduction--author-info)
2. [Environment Setup](#2-environment-setup)
3. [Data Loading & Exploration](#3-data-loading--exploration)
4. [Exploratory Data Analysis (EDA)](#4-exploratory-data-analysis-eda)
5. [Data Preprocessing & Augmentation](#5-data-preprocessing--augmentation)
6. [Model Architecture](#6-model-architecture)
7. [Training Pipeline](#7-training-pipeline)
8. [Inference & Embedding Generation](#8-inference--embedding-generation)
9. [Submission Generation](#9-submission-generation)
10. [Results Visualization](#10-results-visualization)
11. [Conclusion](#11-conclusion)

---

## 📊 List of Plots & Visualizations

| # | Plot Name | Description |
|---|-----------|-------------|
| 1 | Class Distribution Bar Plot | Shows number of images per jaguar identity |
| 2 | Class Distribution Pie Chart | Percentage distribution of top jaguars |
| 3 | Sample Images Grid | Display sample images for each jaguar |
| 4 | Image Size Distribution | Histogram of image dimensions |
| 5 | Image Aspect Ratio Distribution | Distribution of width/height ratios |
| 6 | Sample Augmentations | Before/after augmentation examples |
| 7 | Training Loss Curve | Loss progression during training |
| 8 | Learning Rate Schedule | LR changes over epochs |
| 9 | Validation Accuracy Curve | Accuracy progression |
| 10 | t-SNE Embedding Visualization | 2D projection of jaguar embeddings |
| 11 | UMAP Embedding Visualization | Alternative 2D embedding projection |
| 12 | Similarity Distribution | Distribution of similarity scores |
| 13 | Confusion Matrix Heatmap | Per-class prediction analysis |
| 14 | Top Matches Visualization | Best matching pairs |
| 15 | Hard Negatives Visualization | Most similar different jaguars |

---

```python
# =============================================================================
# CELL 1: INTRODUCTION & AUTHOR INFO
# =============================================================================
```

```markdown
# 🐆 Jaguar Re-Identification Challenge

## Competition Overview
This notebook presents a comprehensive solution for the **Jaguar Re-Identification Challenge**, 
where we develop a computer vision model to identify individual jaguars from wildlife photographs.

## Author Information

### 👨‍💻 [YOUR NAME HERE]

**Machine Learning Engineer | Data Scientist | AI Researcher**

📧 **Email:** your.email@example.com

🔗 **Social Networks:**
- 💼 [LinkedIn](https://www.linkedin.com/in/hammad-zahid-xyz/)
- 🐙 [GitHub](https://github.com/Hamad-Ansari)
- 🐦 [Twitter/X](https://twitter.com/zahid_hamm57652)
- 📊 [Kaggle](https://www.kaggle.com/hammadansari7)


---

### 🎯 Approach Summary:
1. **Backbone:** EfficientNet-B4 / ConvNeXt with pretrained weights
2. **Loss Function:** ArcFace Loss for metric learning
3. **Data Augmentation:** Heavy augmentation to handle intra-class variation
4. **Handling Imbalance:** Weighted sampling + Focal Loss
5. **Inference:** Cosine similarity between embeddings

---
```

```python
# =============================================================================
# CELL 2: ENVIRONMENT SETUP & IMPORTS
# =============================================================================
"""
Environment Setup - Installing and importing all necessary libraries
"""

# Install required packages (uncomment if needed)
# !pip install -q torch torchvision timm albumentations
# !pip install -q pytorch-metric-learning
# !pip install -q umap-learn
# !pip install -q wandb
# !pip install -q tqdm matplotlib seaborn pandas numpy scikit-learn
# !pip install -q opencv-python-headless pillow

import os
import gc
import random
import warnings
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2

# PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, OneCycleLR
from torch.cuda.amp import GradScaler, autocast

# Torchvision
import torchvision.transforms as T
from torchvision import models

# Timm for pretrained models
import timm

# Albumentations for augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Sklearn utilities
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from sklearn.metrics import average_precision_score

# Progress bar
from tqdm.auto import tqdm

# Suppress warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
def set_seed(seed: int = 42):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Using device: {DEVICE}")
print(f"🐍 Python version: {os.sys.version}")
print(f"🔥 PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
```

```python
# =============================================================================
# CELL 3: CONFIGURATION
# =============================================================================
"""
Configuration - All hyperparameters and paths in one place
"""

class CFG:
    """Configuration class for all hyperparameters"""
    
    # Paths
    BASE_PATH = Path("./")  # Update this to your data directory
    TRAIN_DIR = BASE_PATH / "train"
    TEST_DIR = BASE_PATH / "test"
    TRAIN_CSV = BASE_PATH / "train.csv"
    TEST_CSV = BASE_PATH / "test.csv"
    SAMPLE_SUB = BASE_PATH / "sample_submission.csv"
    OUTPUT_DIR = BASE_PATH / "outputs"
    
    # Model
    MODEL_NAME = "tf_efficientnet_b4_ns"  # Options: convnext_base, efficientnet_b4, etc.
    PRETRAINED = True
    EMBEDDING_DIM = 512
    NUM_CLASSES = 31  # Number of unique jaguars
    
    # Training
    EPOCHS = 30
    BATCH_SIZE = 16
    ACCUMULATION_STEPS = 2
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-4
    WARMUP_EPOCHS = 3
    
    # ArcFace parameters
    ARCFACE_S = 30.0  # Scale
    ARCFACE_M = 0.5   # Margin
    
    # Image
    IMG_SIZE = 384
    NUM_WORKERS = 4
    
    # Augmentation
    AUGMENT_PROB = 0.5
    
    # Cross-validation
    N_FOLDS = 5
    SELECTED_FOLDS = [0]  # Which folds to train
    
    # Misc
    SEED = 42
    MIXED_PRECISION = True
    DEBUG = False  # Set to True for quick debugging
    
    @classmethod
    def display(cls):
        """Display all configuration parameters"""
        print("=" * 60)
        print("📋 CONFIGURATION")
        print("=" * 60)
        for key, value in vars(cls).items():
            if not key.startswith('_') and not callable(value):
                print(f"  {key}: {value}")
        print("=" * 60)

# Create output directory
CFG.OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# Display configuration
CFG.display()
```

```python
# =============================================================================
# CELL 4: DATA LOADING
# =============================================================================
"""
Data Loading - Load and examine the dataset files
"""

# Load CSV files
train_df = pd.read_csv(CFG.TRAIN_CSV)
test_df = pd.read_csv(CFG.TEST_CSV)
sample_sub = pd.read_csv(CFG.SAMPLE_SUB)

print("📁 Data Loaded Successfully!")
print("=" * 60)

# Display basic info
print("\n📊 TRAIN DATA:")
print(f"  Shape: {train_df.shape}")
print(f"  Columns: {list(train_df.columns)}")
print(f"  Unique Jaguars: {train_df['ground_truth'].nunique()}")
print(f"\n  Sample rows:")
display(train_df.head(10))

print("\n📊 TEST DATA:")
print(f"  Shape: {test_df.shape}")
print(f"  Columns: {list(test_df.columns)}")
print(f"  Unique Query Images: {test_df['query_image'].nunique()}")
print(f"  Unique Gallery Images: {test_df['gallery_image'].nunique()}")
print(f"\n  Sample rows:")
display(test_df.head(10))

print("\n📊 SAMPLE SUBMISSION:")
print(f"  Shape: {sample_sub.shape}")
print(f"  Columns: {list(sample_sub.columns)}")
print(f"\n  Sample rows:")
display(sample_sub.head(10))
```

```python
# =============================================================================
# CELL 5: EXPLORATORY DATA ANALYSIS (EDA)
# =============================================================================
"""
Exploratory Data Analysis - Understanding the dataset through visualizations
"""

# Create figure with subplots
fig = plt.figure(figsize=(20, 16))

# ============================================================================
# PLOT 1: Class Distribution Bar Plot
# ============================================================================
ax1 = fig.add_subplot(2, 2, 1)
class_counts = train_df['ground_truth'].value_counts()

colors = plt.cm.viridis(np.linspace(0, 1, len(class_counts)))
bars = ax1.barh(class_counts.index, class_counts.values, color=colors)
ax1.set_xlabel('Number of Images', fontsize=12)
ax1.set_ylabel('Jaguar Identity', fontsize=12)
ax1.set_title('📊 Plot 1: Class Distribution - Images per Jaguar', fontsize=14, fontweight='bold')
ax1.invert_yaxis()

# Add value labels
for bar, val in zip(bars, class_counts.values):
    ax1.text(val + 1, bar.get_y() + bar.get_height()/2, str(val), 
             va='center', fontsize=9)

# ============================================================================
# PLOT 2: Class Distribution Pie Chart (Top 10)
# ============================================================================
ax2 = fig.add_subplot(2, 2, 2)
top_10 = class_counts.head(10)
others = class_counts.iloc[10:].sum()
pie_data = list(top_10.values) + [others]
pie_labels = list(top_10.index) + ['Others']

colors_pie = plt.cm.Set3(np.linspace(0, 1, len(pie_data)))
wedges, texts, autotexts = ax2.pie(pie_data, labels=pie_labels, autopct='%1.1f%%',
                                    colors=colors_pie, pctdistance=0.8)
ax2.set_title('📊 Plot 2: Top 10 Jaguars Distribution', fontsize=14, fontweight='bold')

# ============================================================================
# PLOT 3: Class Imbalance Analysis
# ============================================================================
ax3 = fig.add_subplot(2, 2, 3)

# Calculate imbalance metrics
min_count = class_counts.min()
max_count = class_counts.max()
mean_count = class_counts.mean()
median_count = class_counts.median()

stats_text = f"""
📈 Dataset Statistics:
─────────────────────
Total Images: {len(train_df):,}
Total Jaguars: {len(class_counts)}
─────────────────────
Min Images/Class: {min_count}
Max Images/Class: {max_count}
Mean: {mean_count:.1f}
Median: {median_count:.1f}
─────────────────────
Imbalance Ratio: {max_count/min_count:.1f}x
"""

ax3.text(0.1, 0.5, stats_text, transform=ax3.transAxes, fontsize=14,
         verticalalignment='center', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
ax3.axis('off')
ax3.set_title('📊 Plot 3: Dataset Statistics', fontsize=14, fontweight='bold')

# ============================================================================
# PLOT 4: Distribution Histogram
# ============================================================================
ax4 = fig.add_subplot(2, 2, 4)
ax4.hist(class_counts.values, bins=15, color='steelblue', edgecolor='black', alpha=0.7)
ax4.axvline(mean_count, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_count:.1f}')
ax4.axvline(median_count, color='green', linestyle='--', linewidth=2, label=f'Median: {median_count:.1f}')
ax4.set_xlabel('Number of Images', fontsize=12)
ax4.set_ylabel('Number of Jaguars', fontsize=12)
ax4.set_title('📊 Plot 4: Distribution of Images per Jaguar', fontsize=14, fontweight='bold')
ax4.legend()

plt.tight_layout()
plt.savefig(CFG.OUTPUT_DIR / 'eda_class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Plot saved to:", CFG.OUTPUT_DIR / 'eda_class_distribution.png')
```

```python
# =============================================================================
# CELL 6: SAMPLE IMAGES VISUALIZATION
# =============================================================================
"""
Visualize sample images for each jaguar identity
"""

def load_image(path: Path) -> np.ndarray:
    """Load an image and convert to RGB"""
    img = Image.open(path).convert('RGB')
    return np.array(img)

def plot_sample_images(df: pd.DataFrame, img_dir: Path, n_samples: int = 3):
    """
    📊 Plot 5: Sample Images Grid
    Display sample images for each jaguar identity
    """
    unique_jaguars = df['ground_truth'].unique()
    n_jaguars = len(unique_jaguars)
    
    fig, axes = plt.subplots(n_jaguars, n_samples, figsize=(4*n_samples, 3*n_jaguars))
    fig.suptitle('📊 Plot 5: Sample Images for Each Jaguar Identity', 
                 fontsize=16, fontweight='bold', y=1.02)
    
    for i, jaguar in enumerate(sorted(unique_jaguars)):
        jaguar_images = df[df['ground_truth'] == jaguar]['filename'].values
        sample_images = jaguar_images[:n_samples]
        
        for j in range(n_samples):
            ax = axes[i, j] if n_jaguars > 1 else axes[j]
            
            if j < len(sample_images):
                img_path = img_dir / sample_images[j]
                if img_path.exists():
                    img = load_image(img_path)
                    ax.imshow(img)
                    ax.set_title(f'{jaguar}' if j == 0 else '', fontsize=10)
                else:
                    ax.text(0.5, 0.5, 'Image\nNot Found', ha='center', va='center')
            else:
                ax.text(0.5, 0.5, 'No More\nImages', ha='center', va='center')
            
            ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(CFG.OUTPUT_DIR / 'sample_images_grid.png', dpi=100, bbox_inches='tight')
    plt.show()

# Show sample images (limit to first 10 jaguars for display)
sample_df = train_df[train_df['ground_truth'].isin(train_df['ground_truth'].unique()[:10])]
plot_sample_images(sample_df, CFG.TRAIN_DIR, n_samples=4)
```

```python
# =============================================================================
# CELL 7: IMAGE PROPERTIES ANALYSIS
# =============================================================================
"""
Analyze image properties like size, aspect ratio, and file sizes
"""

def analyze_image_properties(df: pd.DataFrame, img_dir: Path, sample_size: int = 200):
    """
    Analyze image properties
    """
    widths, heights, aspect_ratios, file_sizes = [], [], [], []
    
    sample_files = df['filename'].sample(min(sample_size, len(df))).values
    
    for filename in tqdm(sample_files, desc="Analyzing images"):
        img_path = img_dir / filename
        if img_path.exists():
            img = Image.open(img_path)
            w, h = img.size
            widths.append(w)
            heights.append(h)
            aspect_ratios.append(w / h)
            file_sizes.append(os.path.getsize(img_path) / 1024)  # KB
    
    return widths, heights, aspect_ratios, file_sizes

# Analyze training images
print("📊 Analyzing image properties...")
widths, heights, aspect_ratios, file_sizes = analyze_image_properties(train_df, CFG.TRAIN_DIR)

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# ============================================================================
# PLOT 6: Image Width Distribution
# ============================================================================
ax1 = axes[0, 0]
ax1.hist(widths, bins=30, color='coral', edgecolor='black', alpha=0.7)
ax1.axvline(np.mean(widths), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {np.mean(widths):.0f}')
ax1.set_xlabel('Width (pixels)')
ax1.set_ylabel('Frequency')
ax1.set_title('📊 Plot 6: Image Width Distribution', fontweight='bold')
ax1.legend()

# ============================================================================
# PLOT 7: Image Height Distribution
# ============================================================================
ax2 = axes[0, 1]
ax2.hist(heights, bins=30, color='lightgreen', edgecolor='black', alpha=0.7)
ax2.axvline(np.mean(heights), color='green', linestyle='--', linewidth=2,
            label=f'Mean: {np.mean(heights):.0f}')
ax2.set_xlabel('Height (pixels)')
ax2.set_ylabel('Frequency')
ax2.set_title('📊 Plot 7: Image Height Distribution', fontweight='bold')
ax2.legend()

# ============================================================================
# PLOT 8: Aspect Ratio Distribution
# ============================================================================
ax3 = axes[1, 0]
ax3.hist(aspect_ratios, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
ax3.axvline(1.0, color='red', linestyle='--', linewidth=2, label='Square (1:1)')
ax3.set_xlabel('Aspect Ratio (Width/Height)')
ax3.set_ylabel('Frequency')
ax3.set_title('📊 Plot 8: Aspect Ratio Distribution', fontweight='bold')
ax3.legend()

# ============================================================================
# PLOT 9: File Size Distribution
# ============================================================================
ax4 = axes[1, 1]
ax4.hist(file_sizes, bins=30, color='plum', edgecolor='black', alpha=0.7)
ax4.axvline(np.mean(file_sizes), color='purple', linestyle='--', linewidth=2,
            label=f'Mean: {np.mean(file_sizes):.0f} KB')
ax4.set_xlabel('File Size (KB)')
ax4.set_ylabel('Frequency')
ax4.set_title('📊 Plot 9: File Size Distribution', fontweight='bold')
ax4.legend()

plt.tight_layout()
plt.savefig(CFG.OUTPUT_DIR / 'image_properties.png', dpi=150, bbox_inches='tight')
plt.show()

# Print statistics
print("\n📈 Image Property Statistics:")
print(f"  Width:  Min={min(widths)}, Max={max(widths)}, Mean={np.mean(widths):.0f}")
print(f"  Height: Min={min(heights)}, Max={max(heights)}, Mean={np.mean(heights):.0f}")
print(f"  Aspect Ratio: Min={min(aspect_ratios):.2f}, Max={max(aspect_ratios):.2f}")
print(f"  File Size: Min={min(file_sizes):.0f}KB, Max={max(file_sizes):.0f}KB")
```

```python
# =============================================================================
# CELL 8: DATA PREPROCESSING - LABEL ENCODING
# =============================================================================
"""
Encode labels and prepare data for training
"""

# Initialize Label Encoder
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['ground_truth'])

# Create mapping dictionaries
label_to_jaguar = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))
jaguar_to_label = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))

print("🏷️ Label Encoding Complete!")
print(f"  Number of classes: {len(label_encoder.classes_)}")
print("\n📋 Label Mapping:")
for label, jaguar in sorted(label_to_jaguar.items()):
    count = len(train_df[train_df['label'] == label])
    print(f"  {label:2d} → {jaguar:15s} ({count:3d} images)")

# Add full path
train_df['image_path'] = train_df['filename'].apply(lambda x: str(CFG.TRAIN_DIR / x))

# Display updated dataframe
print("\n📊 Updated Training DataFrame:")
display(train_df.head(10))
```

```python
# =============================================================================
# CELL 9: DATA AUGMENTATION
# =============================================================================
"""
Define data augmentation pipelines for training and validation
"""

def get_train_transforms(img_size: int = 384):
    """
    Training augmentation pipeline with heavy augmentation
    to handle intra-class variation
    """
    return A.Compose([
        # Resize
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size, 
                      border_mode=cv2.BORDER_CONSTANT, value=0),
        
        # Geometric transforms
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
        A.Perspective(scale=(0.02, 0.05), p=0.3),
        
        # Color transforms
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=1),
            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=20, p=1),
        ], p=0.5),
        
        # Noise and blur
        A.OneOf([
            A.GaussNoise(var_limit=(10.0, 50.0), p=1),
            A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1),
        ], p=0.3),
        
        A.OneOf([
            A.MotionBlur(blur_limit=5, p=1),
            A.GaussianBlur(blur_limit=5, p=1),
        ], p=0.2),
        
        # Cutout/dropout
        A.CoarseDropout(max_holes=8, max_height=img_size//16, max_width=img_size//16,
                        min_holes=1, min_height=img_size//32, min_width=img_size//32,
                        fill_value=0, p=0.3),
        
        # Normalize and convert
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

def get_valid_transforms(img_size: int = 384):
    """
    Validation transform pipeline (minimal augmentation)
    """
    return A.Compose([
        A.LongestMaxSize(max_size=img_size),
        A.PadIfNeeded(min_height=img_size, min_width=img_size,
                      border_mode=cv2.BORDER_CONSTANT, value=0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

def get_test_transforms(img_size: int = 384):
    """
    Test time augmentation (same as validation)
    """
    return get_valid_transforms(img_size)

# Visualize augmentations
def visualize_augmentations(df: pd.DataFrame, img_dir: Path, n_samples: int = 4):
    """
    📊 Plot 10: Sample Augmentations
    Show original and augmented images side by side
    """
    fig, axes = plt.subplots(n_samples, 5, figsize=(20, 4*n_samples))
    fig.suptitle('📊 Plot 10: Data Augmentation Examples\n(Original → Augmented Versions)', 
                 fontsize=16, fontweight='bold', y=1.02)
    
    transform = get_train_transforms(CFG.IMG_SIZE)
    sample_files = df['filename'].sample(n_samples).values
    
    for i, filename in enumerate(sample_files):
        img_path = img_dir / filename
        if img_path.exists():
            img = cv2.imread(str(img_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Original
            axes[i, 0].imshow(img)
            axes[i, 0].set_title('Original' if i == 0 else '')
            axes[i, 0].axis('off')
            
            # Augmented versions
            for j in range(1, 5):
                augmented = transform(image=img)['image']
                # Denormalize for visualization
                aug_img = augmented.permute(1, 2, 0).numpy()
                aug_img = aug_img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                aug_img = np.clip(aug_img, 0, 1)
                
                axes[i, j].imshow(aug_img)
                axes[i, j].set_title(f'Augmented {j}' if i == 0 else '')
                axes[i, j].axis('off')
    
    plt.tight_layout()
    plt.savefig(CFG.OUTPUT_DIR / 'augmentation_samples.png', dpi=150, bbox_inches='tight')
    plt.show()

# Visualize augmentations
visualize_augmentations(train_df, CFG.TRAIN_DIR, n_samples=4)
```

```python
# =============================================================================
# CELL 10: CUSTOM DATASET CLASS
# =============================================================================
"""
PyTorch Dataset for Jaguar Re-Identification
"""

class JaguarDataset(Dataset):
    """
    Custom Dataset for Jaguar Re-Identification
    """
    def __init__(
        self, 
        df: pd.DataFrame, 
        img_dir: Path,
        transform=None,
        is_test: bool = False
    ):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        
        if not is_test:
            self.labels = self.df['label'].values
            self.filenames = self.df['filename'].values
        else:
            self.filenames = df['filename'].values if 'filename' in df.columns else df.values
    
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, idx):
        filename = self.filenames[idx]
        img_path = self.img_dir / filename
        
        # Load image
        image = cv2.imread(str(img_path))
        if image is None:
            # Handle missing images with a black image
            image = np.zeros((CFG.IMG_SIZE, CFG.IMG_SIZE, 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Apply transforms
        if self.transform:
            image = self.transform(image=image)['image']
        
        if self.is_test:
            return image, filename
        else:
            label = self.labels[idx]
            return image, label
    
    def get_labels(self):
        """Return labels for weighted sampler"""
        return self.labels


class JaguarTestDataset(Dataset):
    """
    Dataset for test images (inference only)
    """
    def __init__(self, image_files: List[str], img_dir: Path, transform=None):
        self.image_files = sorted(list(set(image_files)))
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        filename = self.image_files[idx]
        img_path = self.img_dir / filename
        
        image = cv2.imread(str(img_path))
        if image is None:
            image = np.zeros((CFG.IMG_SIZE, CFG.IMG_SIZE, 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            image = self.transform(image=image)['image']
        
        return image, filename

print("✅ Dataset classes defined successfully!")
```

```python
# =============================================================================
# CELL 11: ARCFACE LOSS IMPLEMENTATION
# =============================================================================
"""
ArcFace Loss for Metric Learning
Ref: https://arxiv.org/abs/1801.07698
"""

class ArcMarginProduct(nn.Module):
    """
    ArcFace: Additive Angular Margin Loss for Deep Face Recognition
    
    Implementation of ArcFace loss which improves discriminability of embeddings
    by adding an angular margin to the softmax loss.
    """
    def __init__(
        self, 
        in_features: int, 
        out_features: int, 
        s: float = 30.0, 
        m: float = 0.50,
        easy_margin: bool = False
    ):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.easy_margin = easy_margin
        
        # Initialize weights
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)
        
        # Precompute values
        self.cos_m = np.cos(m)
        self.sin_m = np.sin(m)
        self.th = np.cos(np.pi - m)
        self.mm = np.sin(np.pi - m) * m
        
    def forward(self, input: torch.Tensor, label: torch.Tensor) -> torch.Tensor:
        # Normalize features and weights
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2).clamp(0, 1))
        
        # cos(theta + m)
        phi = cosine * self.cos_m - sine * self.sin_m
        
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        
        # One-hot encoding
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        
        # Apply margin to target class
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        
        return output


class FocalLoss(nn.Module):
    """
    Focal Loss for handling class imbalance
    Ref: https://arxiv.org/abs/1708.02002
    """
    def __init__(self, gamma: float = 2.0, alpha: Optional[torch.Tensor] = None):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        
    def forward(self, input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        ce_loss = F.cross_entropy(input, target, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        
        if self.alpha is not None:
            alpha_t = self.alpha[target]
            focal_loss = alpha_t * focal_loss
        
        return focal_loss.mean()

print("✅ ArcFace and Focal Loss implemented!")
```

```python
# =============================================================================
# CELL 12: MODEL ARCHITECTURE
# =============================================================================
"""
Jaguar Re-ID Model with Pretrained Backbone and ArcFace Head
"""

class JaguarReIDModel(nn.Module):
    """
    Jaguar Re-Identification Model
    
    Architecture:
    - Pretrained backbone (EfficientNet/ConvNeXt)
    - Global Average Pooling
    - Embedding layer with batch normalization
    - ArcFace classification head
    """
    def __init__(
        self,
        model_name: str = 'tf_efficientnet_b4_ns',
        pretrained: bool = True,
        embedding_dim: int = 512,
        num_classes: int = 31,
        arcface_s: float = 30.0,
        arcface_m: float = 0.5,
        dropout: float = 0.3
    ):
        super().__init__()
        
        # Backbone
        self.backbone = timm.create_model(
            model_name, 
            pretrained=pretrained, 
            num_classes=0,  # Remove classifier
            global_pool='avg'
        )
        
        # Get backbone output features
        in_features = self.backbone.num_features
        
        # Embedding layers
        self.embedding = nn.Sequential(
            nn.Linear(in_features, embedding_dim),
            nn.BatchNorm1d(embedding_dim),
            nn.PReLU(),
            nn.Dropout(dropout),
        )
        
        # ArcFace head
        self.arcface = ArcMarginProduct(
            in_features=embedding_dim,
            out_features=num_classes,
            s=arcface_s,
            m=arcface_m
        )
        
        self.embedding_dim = embedding_dim
        self.num_classes = num_classes
        
    def forward(self, x: torch.Tensor, labels: Optional[torch.Tensor] = None):
        """
        Forward pass
        
        Args:
            x: Input images [B, 3, H, W]
            labels: Class labels [B] (required for training with ArcFace)
            
        Returns:
            If labels provided: ArcFace logits for training
            If no labels: Normalized embeddings for inference
        """
        # Extract features
        features = self.backbone(x)
        
        # Get embeddings
        embeddings = self.embedding(features)
        
        if labels is not None:
            # Training mode: return ArcFace logits
            logits = self.arcface(embeddings, labels)
            return logits, embeddings
        else:
            # Inference mode: return normalized embeddings
            embeddings = F.normalize(embeddings, p=2, dim=1)
            return embeddings
    
    def get_embedding(self, x: torch.Tensor) -> torch.Tensor:
        """Extract normalized embeddings for inference"""
        features = self.backbone(x)
        embeddings = self.embedding(features)
        embeddings = F.normalize(embeddings, p=2, dim=1)
        return embeddings


# Test model creation
print("🏗️ Building model...")
model = JaguarReIDModel(
    model_name=CFG.MODEL_NAME,
    pretrained=CFG.PRETRAINED,
    embedding_dim=CFG.EMBEDDING_DIM,
    num_classes=CFG.NUM_CLASSES,
    arcface_s=CFG.ARCFACE_S,
    arcface_m=CFG.ARCFACE_M
)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✅ Model created successfully!")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Backbone: {CFG.MODEL_NAME}")
print(f"  Embedding dim: {CFG.EMBEDDING_DIM}")

# Test forward pass
model = model.to(DEVICE)
dummy_input = torch.randn(2, 3, CFG.IMG_SIZE, CFG.IMG_SIZE).to(DEVICE)
dummy_labels = torch.tensor([0, 1]).to(DEVICE)

with torch.no_grad():
    logits, embeddings = model(dummy_input, dummy_labels)
    print(f"\n  Test forward pass:")
    print(f"    Input shape: {dummy_input.shape}")
    print(f"    Logits shape: {logits.shape}")
    print(f"    Embeddings shape: {embeddings.shape}")

# Clean up
del model, dummy_input, dummy_labels
torch.cuda.empty_cache()
gc.collect()
```

```python
# =============================================================================
# CELL 13: WEIGHTED SAMPLER FOR CLASS IMBALANCE
# =============================================================================
"""
Create weighted sampler to handle class imbalance during training
"""

def create_weighted_sampler(labels: np.ndarray) -> WeightedRandomSampler:
    """
    Create a weighted random sampler for balanced training
    
    Args:
        labels: Array of class labels
        
    Returns:
        WeightedRandomSampler instance
    """
    class_counts = Counter(labels)
    num_samples = len(labels)
    
    # Calculate weight for each class (inverse frequency)
    class_weights = {cls: num_samples / count for cls, count in class_counts.items()}
    
    # Assign weight to each sample
    sample_weights = [class_weights[label] for label in labels]
    sample_weights = torch.tensor(sample_weights, dtype=torch.float)
    
    # Create sampler
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    return sampler

# Calculate class weights for focal loss
def calculate_class_weights(labels: np.ndarray) -> torch.Tensor:
    """Calculate class weights based on inverse frequency"""
    class_counts = Counter(labels)
    total = len(labels)
    num_classes = len(class_counts)
    
    weights = torch.zeros(num_classes)
    for cls, count in class_counts.items():
        weights[cls] = total / (num_classes * count)
    
    return weights

# Display class balance info
class_counts = train_df['label'].value_counts().sort_index()
print("⚖️ Class Balance Analysis:")
print(f"  Most represented: {label_to_jaguar[class_counts.idxmax()]} ({class_counts.max()} images)")
print(f"  Least represented: {label_to_jaguar[class_counts.idxmin()]} ({class_counts.min()} images)")
print(f"  Imbalance ratio: {class_counts.max() / class_counts.min():.2f}x")
```

```python
# =============================================================================
# CELL 14: TRAINING UTILITIES
# =============================================================================
"""
Training helper functions
"""

class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def calculate_accuracy(logits: torch.Tensor, labels: torch.Tensor) -> float:
    """Calculate classification accuracy"""
    preds = logits.argmax(dim=1)
    correct = (preds == labels).sum().item()
    return correct / len(labels)


def save_checkpoint(model, optimizer, scheduler, epoch, fold, loss, path):
    """Save model checkpoint"""
    checkpoint = {
        'epoch': epoch,
        'fold': fold,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'loss': loss,
    }
    torch.save(checkpoint, path)


def load_checkpoint(model, path):
    """Load model checkpoint"""
    checkpoint = torch.load(path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model, checkpoint['epoch'], checkpoint['loss']

print("✅ Training utilities defined!")
```

```python
# =============================================================================
# CELL 15: TRAINING LOOP
# =============================================================================
"""
Main training loop with mixed precision and gradient accumulation
"""

def train_one_epoch(
    model: nn.Module,
    train_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: nn.Module,
    scheduler,
    epoch: int,
    device: torch.device,
    scaler: GradScaler,
    accumulation_steps: int = 1
) -> Tuple[float, float]:
    """
    Train for one epoch
    """
    model.train()
    
    loss_meter = AverageMeter()
    acc_meter = AverageMeter()
    
    pbar = tqdm(train_loader, desc=f'Train Epoch {epoch}')
    
    optimizer.zero_grad()
    
    for step, (images, labels) in enumerate(pbar):
        images = images.to(device)
        labels = labels.to(device)
        
        # Mixed precision forward pass
        with autocast(enabled=CFG.MIXED_PRECISION):
            logits, embeddings = model(images, labels)
            loss = criterion(logits, labels)
            loss = loss / accumulation_steps
        
        # Backward pass with gradient scaling
        scaler.scale(loss).backward()
        
        # Gradient accumulation
        if (step + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            
            if scheduler is not None:
                scheduler.step()
        
        # Calculate metrics
        acc = calculate_accuracy(logits.detach(), labels)
        
        loss_meter.update(loss.item() * accumulation_steps, images.size(0))
        acc_meter.update(acc, images.size(0))
        
        # Update progress bar
        pbar.set_postfix({
            'loss': f'{loss_meter.avg:.4f}',
            'acc': f'{acc_meter.avg:.4f}',
            'lr': f'{optimizer.param_groups[0]["lr"]:.2e}'
        })
    
    return loss_meter.avg, acc_meter.avg


@torch.no_grad()
def validate(
    model: nn.Module,
    valid_loader: DataLoader,
    criterion: nn.Module,
    device: torch.device
) -> Tuple[float, float]:
    """
    Validate model
    """
    model.eval()
    
    loss_meter = AverageMeter()
    acc_meter = AverageMeter()
    
    pbar = tqdm(valid_loader, desc='Validation')
    
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)
        
        with autocast(enabled=CFG.MIXED_PRECISION):
            logits, embeddings = model(images, labels)
            loss = criterion(logits, labels)
        
        acc = calculate_accuracy(logits.detach(), labels)
        
        loss_meter.update(loss.item(), images.size(0))
        acc_meter.update(acc, images.size(0))
        
        pbar.set_postfix({
            'loss': f'{loss_meter.avg:.4f}',
            'acc': f'{acc_meter.avg:.4f}'
        })
    
    return loss_meter.avg, acc_meter.avg

print("✅ Training loop defined!")
```

```python
# =============================================================================
# CELL 16: CROSS-VALIDATION TRAINING
# =============================================================================
"""
Full training pipeline with K-Fold Cross-Validation
"""

def train_fold(fold: int, train_df: pd.DataFrame):
    """
    Train a single fold
    """
    print(f"\n{'='*60}")
    print(f"🔄 TRAINING FOLD {fold}")
    print(f"{'='*60}")
    
    # Split data
    skf = StratifiedKFold(n_splits=CFG.N_FOLDS, shuffle=True, random_state=CFG.SEED)
    
    for fold_idx, (train_idx, valid_idx) in enumerate(skf.split(train_df, train_df['label'])):
        if fold_idx == fold:
            break
    
    train_data = train_df.iloc[train_idx].reset_index(drop=True)
    valid_data = train_df.iloc[valid_idx].reset_index(drop=True)
    
    print(f"  Train samples: {len(train_data)}")
    print(f"  Valid samples: {len(valid_data)}")
    
    # Create datasets
    train_dataset = JaguarDataset(
        train_data, 
        CFG.TRAIN_DIR,
        transform=get_train_transforms(CFG.IMG_SIZE)
    )
    
    valid_dataset = JaguarDataset(
        valid_data,
        CFG.TRAIN_DIR,
        transform=get_valid_transforms(CFG.IMG_SIZE)
    )
    
    # Create weighted sampler
    sampler = create_weighted_sampler(train_data['label'].values)
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.BATCH_SIZE,
        sampler=sampler,
        num_workers=CFG.NUM_WORKERS,
        pin_memory=True,
        drop_last=True
    )
    
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=CFG.BATCH_SIZE * 2,
        shuffle=False,
        num_workers=CFG.NUM_WORKERS,
        pin_memory=True,
        drop_last=False
    )
    
    # Initialize model
    model = JaguarReIDModel(
        model_name=CFG.MODEL_NAME,
        pretrained=CFG.PRETRAINED,
        embedding_dim=CFG.EMBEDDING_DIM,
        num_classes=CFG.NUM_CLASSES,
        arcface_s=CFG.ARCFACE_S,
        arcface_m=CFG.ARCFACE_M
    ).to(DEVICE)
    
    # Loss function with class weights
    class_weights = calculate_class_weights(train_data['label'].values).to(DEVICE)
    criterion = FocalLoss(gamma=2.0, alpha=class_weights)
    
    # Optimizer
    optimizer = AdamW(
        model.parameters(),
        lr=CFG.LEARNING_RATE,
        weight_decay=CFG.WEIGHT_DECAY
    )
    
    # Scheduler
    num_training_steps = len(train_loader) * CFG.EPOCHS
    scheduler = OneCycleLR(
        optimizer,
        max_lr=CFG.LEARNING_RATE * 10,
        total_steps=num_training_steps,
        pct_start=0.1,
        anneal_strategy='cos'
    )
    
    # Mixed precision scaler
    scaler = GradScaler(enabled=CFG.MIXED_PRECISION)
    
    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'valid_loss': [],
        'valid_acc': [],
        'lr': []
    }
    
    best_loss = float('inf')
    best_acc = 0
    
    # Training loop
    for epoch in range(1, CFG.EPOCHS + 1):
        print(f"\n📅 Epoch {epoch}/{CFG.EPOCHS}")
        
        # Train
        train_loss, train_acc = train_one_epoch(
            model, train_loader, optimizer, criterion,
            scheduler, epoch, DEVICE, scaler, CFG.ACCUMULATION_STEPS
        )
        
        # Validate
        valid_loss, valid_acc = validate(
            model, valid_loader, criterion, DEVICE
        )
        
        # Log history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['valid_loss'].append(valid_loss)
        history['valid_acc'].append(valid_acc)
        history['lr'].append(optimizer.param_groups[0]['lr'])
        
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Valid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc:.4f}")
        
        # Save best model
        if valid_loss < best_loss:
            best_loss = valid_loss
            best_acc = valid_acc
            save_checkpoint(
                model, optimizer, scheduler, epoch, fold, valid_loss,
                CFG.OUTPUT_DIR / f'best_model_fold{fold}.pth'
            )
            print(f"  ✅ Best model saved! (Loss: {best_loss:.4f}, Acc: {best_acc:.4f})")
        
        # Early stopping check (optional)
        if CFG.DEBUG and epoch >= 3:
            print("  ⚠️ Debug mode: stopping early")
            break
    
    print(f"\n🏆 Fold {fold} Complete!")
    print(f"  Best Valid Loss: {best_loss:.4f}")
    print(f"  Best Valid Acc: {best_acc:.4f}")
    
    # Clean up
    del model, optimizer, scheduler, train_loader, valid_loader
    torch.cuda.empty_cache()
    gc.collect()
    
    return history

# Train selected folds
all_histories = {}
for fold in CFG.SELECTED_FOLDS:
    history = train_fold(fold, train_df)
    all_histories[fold] = history
```

```python
# =============================================================================
# CELL 17: TRAINING VISUALIZATION
# =============================================================================
"""
Visualize training progress with loss and accuracy curves
"""

def plot_training_history(histories: Dict[int, dict]):
    """
    📊 Plot 11-13: Training History Curves
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    colors = plt.cm.Set2(np.linspace(0, 1, len(histories)))
    
    for idx, (fold, history) in enumerate(histories.items()):
        color = colors[idx]
        epochs = range(1, len(history['train_loss']) + 1)
        
        # ============================================================================
        # PLOT 11: Training Loss Curve
        # ============================================================================
        axes[0, 0].plot(epochs, history['train_loss'], 
                       label=f'Fold {fold} Train', color=color, linestyle='-', linewidth=2)
        axes[0, 0].plot(epochs, history['valid_loss'], 
                       label=f'Fold {fold} Valid', color=color, linestyle='--', linewidth=2)
    
    axes[0, 0].set_xlabel('Epoch', fontsize=12)
    axes[0, 0].set_ylabel('Loss', fontsize=12)
    axes[0, 0].set_title('📊 Plot 11: Training & Validation Loss', fontsize=14, fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # ============================================================================
    # PLOT 12: Accuracy Curve
    # ============================================================================
    for idx, (fold, history) in enumerate(histories.items()):
        color = colors[idx]
        epochs = range(1, len(history['train_acc']) + 1)
        
        axes[0, 1].plot(epochs, history['train_acc'], 
                       label=f'Fold {fold} Train', color=color, linestyle='-', linewidth=2)
        axes[0, 1].plot(epochs, history['valid_acc'], 
                       label=f'Fold {fold} Valid', color=color, linestyle='--', linewidth=2)
    
    axes[0, 1].set_xlabel('Epoch', fontsize=12)
    axes[0, 1].set_ylabel('Accuracy', fontsize=12)
    axes[0, 1].set_title('📊 Plot 12: Training & Validation Accuracy', fontsize=14, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].set_ylim([0, 1])
    
    # ============================================================================
    # PLOT 13: Learning Rate Schedule
    # ============================================================================
    for idx, (fold, history) in enumerate(histories.items()):
        color = colors[idx]
        epochs = range(1, len(history['lr']) + 1)
        
        axes[1, 0].plot(epochs, history['lr'], 
                       label=f'Fold {fold}', color=color, linewidth=2)
    
    axes[1, 0].set_xlabel('Epoch', fontsize=12)
    axes[1, 0].set_ylabel('Learning Rate', fontsize=12)
    axes[1, 0].set_title('📊 Plot 13: Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].set_yscale('log')
    
    # ============================================================================
    # PLOT 14: Loss vs Accuracy
    # ============================================================================
    for idx, (fold, history) in enumerate(histories.items()):
        color = colors[idx]
        axes[1, 1].scatter(history['valid_loss'], history['valid_acc'], 
                          c=[color] * len(history['valid_loss']), 
                          label=f'Fold {fold}', alpha=0.7, s=50)
    
    axes[1, 1].set_xlabel('Validation Loss', fontsize=12)
    axes[1, 1].set_ylabel('Validation Accuracy', fontsize=12)
    axes[1, 1].set_title('📊 Plot 14: Loss vs Accuracy Trade-off', fontsize=14, fontweight='bold')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(CFG.OUTPUT_DIR / 'training_history.png', dpi=150, bbox_inches='tight')
    plt.show()

# Plot training history
if all_histories:
    plot_training_history(all_histories)
```

```python
# =============================================================================
# CELL 18: INFERENCE - EXTRACT EMBEDDINGS
# =============================================================================
"""
Extract embeddings for test images
"""

@torch.no_grad()
def extract_embeddings(
    model: nn.Module,
    dataloader: DataLoader,
    device: torch.device
) -> Tuple[np.ndarray, List[str]]:
    """
    Extract embeddings for all images in dataloader
    """
    model.eval()
    
    all_embeddings = []
    all_filenames = []
    
    pbar = tqdm(dataloader, desc='Extracting embeddings')
    
    for images, filenames in pbar:
        images = images.to(device)
        
        with autocast(enabled=CFG.MIXED_PRECISION):
            embeddings = model.get_embedding(images)
        
        all_embeddings.append(embeddings.cpu().numpy())
        all_filenames.extend(filenames)
    
    embeddings = np.vstack(all_embeddings)
    
    return embeddings, all_filenames


def load_best_model(fold: int = 0) -> nn.Module:
    """Load the best model from a specific fold"""
    model = JaguarReIDModel(
        model_name=CFG.MODEL_NAME,
        pretrained=False,
        embedding_dim=CFG.EMBEDDING_DIM,
        num_classes=CFG.NUM_CLASSES,
        arcface_s=CFG.ARCFACE_S,
        arcface_m=CFG.ARCFACE_M
    ).to(DEVICE)
    
    checkpoint_path = CFG.OUTPUT_DIR / f'best_model_fold{fold}.pth'
    if checkpoint_path.exists():
        model, _, _ = load_checkpoint(model, checkpoint_path)
        print(f"✅ Loaded model from {checkpoint_path}")
    else:
        print(f"⚠️ No checkpoint found at {checkpoint_path}")
    
    return model


# Get unique test images
test_images = list(set(test_df['query_image'].tolist() + test_df['gallery_image'].tolist()))
test_images = sorted(test_images)
print(f"📊 Total unique test images: {len(test_images)}")

# Create test dataset and dataloader
test_dataset = JaguarTestDataset(
    image_files=test_images,
    img_dir=CFG.TEST_DIR,
    transform=get_test_transforms(CFG.IMG_SIZE)
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CFG.BATCH_SIZE * 2,
    shuffle=False,
    num_workers=CFG.NUM_WORKERS,
    pin_memory=True
)

# Load best model and extract embeddings
model = load_best_model(fold=CFG.SELECTED_FOLDS[0])
test_embeddings, test_filenames = extract_embeddings(model, test_loader, DEVICE)

# Create filename to embedding mapping
embedding_dict = dict(zip(test_filenames, test_embeddings))

print(f"\n✅ Embeddings extracted!")
print(f"  Shape: {test_embeddings.shape}")
print(f"  Files: {len(test_filenames)}")
```

```python
# =============================================================================
# CELL 19: EMBEDDING VISUALIZATION
# =============================================================================
"""
Visualize embeddings using t-SNE and UMAP
"""

# Also extract train embeddings for visualization
train_dataset_viz = JaguarDataset(
    train_df,
    CFG.TRAIN_DIR,
    transform=get_valid_transforms(CFG.IMG_SIZE)
)

train_loader_viz = DataLoader(
    train_dataset_viz,
    batch_size=CFG.BATCH_SIZE * 2,
    shuffle=False,
    num_workers=CFG.NUM_WORKERS,
    pin_memory=True
)

# Extract train embeddings
@torch.no_grad()
def extract_train_embeddings(model, dataloader, device):
    """Extract embeddings and labels for training data"""
    model.eval()
    
    all_embeddings = []
    all_labels = []
    
    for images, labels in tqdm(dataloader, desc='Extracting train embeddings'):
        images = images.to(device)
        embeddings = model.get_embedding(images)
        all_embeddings.append(embeddings.cpu().numpy())
        all_labels.extend(labels.numpy())
    
    return np.vstack(all_embeddings), np.array(all_labels)

train_embeddings, train_labels = extract_train_embeddings(model, train_loader_viz, DEVICE)

# t-SNE visualization
print("\n🔄 Computing t-SNE...")
tsne = TSNE(n_components=2, random_state=CFG.SEED, perplexity=30, n_iter=1000)
train_tsne = tsne.fit_transform(train_embeddings)

# Plot t-SNE
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# ============================================================================
# PLOT 15: t-SNE Embedding Visualization
# ============================================================================
scatter = axes[0].scatter(
    train_tsne[:, 0], train_tsne[:, 1],
    c=train_labels, cmap='tab20', alpha=0.7, s=30
)
axes[0].set_xlabel('t-SNE Dimension 1', fontsize=12)
axes[0].set_ylabel('t-SNE Dimension 2', fontsize=12)
axes[0].set_title('📊 Plot 15: t-SNE Visualization of Jaguar Embeddings', 
                  fontsize=14, fontweight='bold')

# Add colorbar
cbar = plt.colorbar(scatter, ax=axes[0])
cbar.set_label('Jaguar ID', fontsize=11)

# ============================================================================
# PLOT 16: t-SNE with Selected Jaguars Highlighted
# ============================================================================
# Highlight a few specific jaguars
selected_jaguars = [0, 5, 10, 15, 20]
colors_selected = ['red', 'blue', 'green', 'orange', 'purple']

axes[1].scatter(train_tsne[:, 0], train_tsne[:, 1], c='lightgray', alpha=0.3, s=20)

for jaguar_id, color in zip(selected_jaguars, colors_selected):
    mask = train_labels == jaguar_id
    jaguar_name = label_to_jaguar.get(jaguar_id, f'ID {jaguar_id}')
    axes[1].scatter(
        train_tsne[mask, 0], train_tsne[mask, 1],
        c=color, label=jaguar_name, s=50, alpha=0.8, edgecolors='black'
    )

axes[1].set_xlabel('t-SNE Dimension 1', fontsize=12)
axes[1].set_ylabel('t-SNE Dimension 2', fontsize=12)
axes[1].set_title('📊 Plot 16: Selected Jaguars Highlighted', fontsize=14, fontweight='bold')
axes[1].legend(loc='upper right')

plt.tight_layout()
plt.savefig(CFG.OUTPUT_DIR / 'embedding_visualization.png', dpi=150, bbox_inches='tight')
plt.show()
```

```python
# =============================================================================
# CELL 20: SIMILARITY COMPUTATION
# =============================================================================
"""
Compute pairwise similarity scores for test image pairs
"""

def compute_cosine_similarity(emb1: np.ndarray, emb2: np.ndarray) -> float:
    """Compute cosine similarity between two embeddings"""
    return np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))


def compute_all_similarities(
    test_df: pd.DataFrame,
    embedding_dict: Dict[str, np.ndarray]
) -> pd.DataFrame:
    """
    Compute similarity scores for all test pairs
    """
    print("🔄 Computing pairwise similarities...")
    
    similarities = []
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
        query_img = row['query_image']
        gallery_img = row['gallery_image']
        
        if query_img in embedding_dict and gallery_img in embedding_dict:
            emb1 = embedding_dict[query_img]
            emb2 = embedding_dict[gallery_img]
            sim = compute_cosine_similarity(emb1, emb2)
            # Convert to probability-like score (0 to 1)
            sim = (sim + 1) / 2  # Shift from [-1,1] to [0,1]
        else:
            sim = 0.5  # Default for missing images
        
        similarities.append(sim)
    
    test_df['similarity'] = similarities
    
    return test_df


# Compute similarities
test_df_with_sim = compute_all_similarities(test_df.copy(), embedding_dict)

print("\n✅ Similarity computation complete!")
print(f"  Total pairs: {len(test_df_with_sim):,}")
print(f"  Similarity stats:")
print(f"    Min: {test_df_with_sim['similarity'].min():.4f}")
print(f"    Max: {test_df_with_sim['similarity'].max():.4f}")
print(f"    Mean: {test_df_with_sim['similarity'].mean():.4f}")
print(f"    Std: {test_df_with_sim['similarity'].std():.4f}")
```

```python
# =============================================================================
# CELL 21: SIMILARITY DISTRIBUTION VISUALIZATION
# =============================================================================
"""
Visualize the distribution of similarity scores
"""

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ============================================================================
# PLOT 17: Similarity Score Distribution
# ============================================================================
axes[0].hist(test_df_with_sim['similarity'], bins=50, color='steelblue', 
             edgecolor='black', alpha=0.7)
axes[0].axvline(test_df_with_sim['similarity'].mean(), color='red', 
               linestyle='--', linewidth=2, label=f"Mean: {test_df_with_sim['similarity'].mean():.3f}")
axes[0].axvline(test_df_with_sim['similarity'].median(), color='green',
               linestyle='--', linewidth=2, label=f"Median: {test_df_with_sim['similarity'].median():.3f}")
axes[0].set_xlabel('Similarity Score', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('📊 Plot 17: Distribution of Similarity Scores', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# ============================================================================
# PLOT 18: Similarity Score Box Plot by Score Range
# ============================================================================
# Create score bins
test_df_with_sim['score_bin'] = pd.cut(test_df_with_sim['similarity'], 
                                        bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0],
                                        labels=['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0'])
score_bin_counts = test_df_with_sim['score_bin'].value_counts().sort_index()

axes[1].bar(score_bin_counts.index.astype(str), score_bin_counts.values, 
           color=plt.cm.RdYlGn(np.linspace(0.2, 0.8, 5)), edgecolor='black')
axes[1].set_xlabel('Similarity Score Range', fontsize=12)
axes[1].set_ylabel('Number of Pairs', fontsize=12)
axes[1].set_title('📊 Plot 18: Pairs Distribution by Similarity Range', fontsize=14, fontweight='bold')

# Add count labels
for i, (idx, val) in enumerate(zip(score_bin_counts.index, score_bin_counts.values)):
    axes[1].text(i, val + 500, f'{val:,}', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig(CFG.OUTPUT_DIR / 'similarity_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
```

```python
# =============================================================================
# CELL 22: TOP MATCHES VISUALIZATION
# =============================================================================
"""
Visualize top matching pairs and potential hard negatives
"""

def visualize_top_matches(df: pd.DataFrame, img_dir: Path, n_pairs: int = 5):
    """
    📊 Plot 19: Top Matching Pairs
    Show the most similar image pairs
    """
    # Sort by similarity
    top_matches = df.nlargest(n_pairs, 'similarity')
    
    fig, axes = plt.subplots(n_pairs, 2, figsize=(12, 4*n_pairs))
    fig.suptitle('📊 Plot 19: Top Matching Pairs (Highest Similarity)', 
                 fontsize=16, fontweight='bold', y=1.02)
    
    for i, (_, row) in enumerate(top_matches.iterrows()):
        query_path = img_dir / row['query_image']
        gallery_path = img_dir / row['gallery_image']
        
        if query_path.exists():
            img1 = load_image(query_path)
            axes[i, 0].imshow(img1)
        axes[i, 0].set_title(f"Query: {row['query_image']}", fontsize=10)
        axes[i, 0].axis('off')
        
        if gallery_path.exists():
            img2 = load_image(gallery_path)
            axes[i, 1].imshow(img2)
        axes[i, 1].set_title(f"Gallery: {row['gallery_image']}\nSimilarity: {row['similarity']:.4f}", 
                            fontsize=10)
        axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig(CFG.OUTPUT_DIR / 'top_matches.png', dpi=150, bbox_inches='tight')
    plt.show()


def visualize_low_matches(df: pd.DataFrame, img_dir: Path, n_pairs: int = 5):
    """
    📊 Plot 20: Lowest Matching Pairs (Likely Different Jaguars)
    """
    low_matches = df.nsmallest(n_pairs, 'similarity')
    
    fig, axes = plt.subplots(n_pairs, 2, figsize=(12, 4*n_pairs))
    fig.suptitle('📊 Plot 20: Lowest Matching Pairs (Likely Different Jaguars)', 
                 fontsize=16, fontweight='bold', y=1.02)
    
    for i, (_, row) in enumerate(low_matches.iterrows()):
        query_path = img_dir / row['query_image']
        gallery_path = img_dir / row['gallery_image']
        
        if query_path.exists():
            img1 = load_image(query_path)
            axes[i, 0].imshow(img1)
        axes[i, 0].set_title(f"Query: {row['query_image']}", fontsize=10)
        axes[i, 0].axis('off')
        
        if gallery_path.exists():
            img2 = load_image(gallery_path)
            axes[i, 1].imshow(img2)
        axes[i, 1].set_title(f"Gallery: {row['gallery_image']}\nSimilarity: {row['similarity']:.4f}", 
                            fontsize=10)
        axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig(CFG.OUTPUT_DIR / 'low_matches.png', dpi=150, bbox_inches='tight')
    plt.show()

# Visualize matches
visualize_top_matches(test_df_with_sim, CFG.TEST_DIR, n_pairs=5)
visualize_low_matches(test_df_with_sim, CFG.TEST_DIR, n_pairs=5)
```

```python
# =============================================================================
# CELL 23: GENERATE SUBMISSION FILE
# =============================================================================
"""
Generate the final submission file
"""

# Create submission dataframe
submission = pd.DataFrame({
    'row_id': test_df_with_sim['row_id'],
    'similarity': test_df_with_sim['similarity']
})

# Ensure proper formatting
submission['row_id'] = submission['row_id'].astype(int)
submission['similarity'] = submission['similarity'].astype(float)

# Clip values to valid range
submission['similarity'] = submission['similarity'].clip(0.0, 1.0)

# Sort by row_id
submission = submission.sort_values('row_id').reset_index(drop=True)

# Save submission
submission_path = CFG.OUTPUT_DIR / 'submission.csv'
submission.to_csv(submission_path, index=False)

print("✅ Submission file generated!")
print(f"  Path: {submission_path}")
print(f"  Shape: {submission.shape}")
print(f"\n📊 Submission Preview:")
display(submission.head(10))
display(submission.tail(10))

# Validate submission
print(f"\n🔍 Submission Validation:")
print(f"  Row IDs match: {(submission['row_id'] == test_df['row_id']).all()}")
print(f"  Valid range: {(submission['similarity'] >= 0).all() and (submission['similarity'] <= 1).all()}")
print(f"  No NaN values: {submission['similarity'].notna().all()}")
```

```python
# =============================================================================
# CELL 24: FINAL SUMMARY
# =============================================================================
"""
Print final summary of the solution
"""

print("=" * 70)
print("🏆 JAGUAR RE-IDENTIFICATION CHALLENGE - SOLUTION SUMMARY")
print("=" * 70)

summary = f"""
📊 DATASET STATISTICS:
   • Training images: {len(train_df):,}
   • Unique jaguars: {train_df['ground_truth'].nunique()}
   • Test image pairs: {len(test_df):,}
   • Unique test images: {len(test_images)}

🏗️ MODEL ARCHITECTURE:
   • Backbone: {CFG.MODEL_NAME}
   • Embedding dimension: {CFG.EMBEDDING_DIM}
   • Loss function: ArcFace + Focal Loss
   • ArcFace scale (s): {CFG.ARCFACE_S}
   • ArcFace margin (m): {CFG.ARCFACE_M}

⚙️ TRAINING CONFIGURATION:
   • Image size: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}
   • Batch size: {CFG.BATCH_SIZE}
   • Epochs: {CFG.EPOCHS}
   • Learning rate: {CFG.LEARNING_RATE}
   • Folds trained: {CFG.SELECTED_FOLDS}

📈 OUTPUT FILES:
   • Model checkpoints: {CFG.OUTPUT_DIR}/best_model_fold*.pth
   • Training plots: {CFG.OUTPUT_DIR}/*.png
   • Submission: {CFG.OUTPUT_DIR}/submission.csv

📋 SUBMISSION STATISTICS:
   • Total predictions: {len(submission):,}
   • Similarity range: [{submission['similarity'].min():.4f}, {submission['similarity'].max():.4f}]
   • Mean similarity: {submission['similarity'].mean():.4f}
"""

print(summary)
print("=" * 70)
print("✅ Notebook execution complete!")
print("=" * 70)
```

```python
# =============================================================================
# CELL 25: PLOTS SUMMARY
# =============================================================================
"""
Summary of all visualization plots generated
"""

plots_summary = """
╔════════════════════════════════════════════════════════════════════════════╗
║                       📊 PLOTS SUMMARY                                      ║
╠════════════════════════════════════════════════════════════════════════════╣
║                                                                            ║
║  EXPLORATORY DATA ANALYSIS (EDA):                                          ║
║  ─────────────────────────────────────────────────────────────────────────║
║  1.  Class Distribution Bar Plot        - Images per jaguar identity      ║
║  2.  Class Distribution Pie Chart       - Top 10 jaguars percentage       ║
║  3.  Dataset Statistics                 - Key metrics summary             ║
║  4.  Distribution Histogram             - Images per class histogram      ║
║  5.  Sample Images Grid                 - Examples for each jaguar        ║
║  6.  Image Width Distribution           - Width histogram                 ║
║  7.  Image Height Distribution          - Height histogram                ║
║  8.  Aspect Ratio Distribution          - Width/Height ratio              ║
║  9.  File Size Distribution             - Image file sizes                ║
║  10. Augmentation Samples               - Before/after augmentation       ║
║                                                                            ║
║  TRAINING VISUALIZATION:                                                   ║
║  ─────────────────────────────────────────────────────────────────────────║
║  11. Training Loss Curve                - Loss over epochs                ║
║  12. Accuracy Curve                     - Train/valid accuracy            ║
║  13. Learning Rate Schedule             - LR changes over training        ║
║  14. Loss vs Accuracy Trade-off         - Scatter plot                    ║
║                                                                            ║
║  EMBEDDING ANALYSIS:                                                       ║
║  ─────────────────────────────────────────────────────────────────────────║
║  15. t-SNE Visualization                - 2D embedding projection         ║
║  16. Selected Jaguars Highlighted       - Specific classes highlighted    ║
║                                                                            ║
║  INFERENCE RESULTS:                                                        ║
║  ─────────────────────────────────────────────────────────────────────────║
║  17. Similarity Score Distribution      - Histogram of predictions        ║
║  18. Pairs by Similarity Range          - Binned distribution             ║
║  19. Top Matching Pairs                 - Highest similarity examples     ║
║  20. Lowest Matching Pairs              - Lowest similarity examples      ║
║                                                                            ║
╚════════════════════════════════════════════════════════════════════════════╝
"""

print(plots_summary)
```

---

## 📝 Author Information Template

```markdown
# 👨‍💻 About the Author

## [YOUR FULL NAME]
**Machine Learning Engineer | Data Scientist | AI Researcher**

### 📫 Contact Information
- 📧 **Email:** your.email@example.com
- 📍 **Location:** Your City, Country

### 🔗 Social Networks & Profiles

| Platform | Link | Description |
|----------|------|-------------|
| 💼 LinkedIn | [linkedin.com/in/your-profile](https://linkedin.com/in/your-profile) | Professional network |
| 🐙 GitHub | [github.com/your-username](https://github.com/your-username) | Code repositories |
| 📊 Kaggle | [kaggle.com/your-username](https://kaggle.com/your-username) | Competition profile |
| 🐦 Twitter/X | [twitter.com/your-handle](https://twitter.com/your-handle) | Tech updates |
| 🌐 Portfolio | [your-website.com](https://your-website.com) | Personal website |
| 📝 Medium | [medium.com/@your-username](https://medium.com/@your-username) | Technical blog |
| 🎓 Google Scholar | [scholar.google.com/citations?user=YOUR_ID](https://scholar.google.com) | Publications |
| 📹 YouTube | [youtube.com/@your-channel](https://youtube.com/@your-channel) | Tutorial videos |

### 🏆 Achievements
- [List your relevant achievements]
- [Kaggle competitions, rankings, etc.]
- [Publications, certifications, etc.]

### 💡 Expertise
- Computer Vision & Image Classification
- Deep Learning & Neural Networks
- Wildlife Conservation Technology
- Re-Identification Systems
- Metric Learning
```

---

## 🎯 Quick Start Guide

To run this notebook:

1. **Set up your environment:**
```bash
pip install torch torchvision timm albumentations pytorch-metric-learning scikit-learn pandas numpy matplotlib seaborn tqdm opencv-python-headless pillow
```

2. **Update the paths in CFG class** to point to your data directory

3. **Run cells sequentially** - each cell is designed to work independently

4. **For quick testing**, set `CFG.DEBUG = True` to run with fewer epochs

---

This comprehensive notebook provides a professional, end-to-end solution for the Jaguar Re-Identification Challenge! 🐆